In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/california-housing-prices/housing.csv


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("camnugent/california-housing-prices")

# print("Path to dataset files:", path)



In [4]:
housing = pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")

In [5]:
housing.drop("ocean_proximity" , inplace=True , axis = 1)
housing = housing.dropna(subset="total_bedrooms")


In [6]:
np.random.seed(42)
x = housing.drop("median_house_value" , axis = 1)
y = housing["median_house_value"]

housing.isna().sum()




longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split 
x_train , x_test , y_train , y_test = train_test_split(x, y , test_size= 0.2)

In [8]:
from sklearn.linear_model import Ridge

model = Ridge()
model.fit(x_train , y_train)



Ridge()

In [9]:
model.score(x_train , y_train)

0.6360185691316951

In [10]:
model.score(x_test , y_test)

0.6400858817984223

# Lasso 

In [11]:
from sklearn.linear_model import Lasso 

model1 = Lasso()

model1.fit(x_train , y_train)

Lasso()

In [12]:
model1.score(x_train , y_train)

0.636018572207647

In [13]:
model1.score(x_test , y_test)

0.6400862985932623